# 3.c RAG in an agent

In this notebook you will see
- How to give the RAG tool to an agent (not to redefine the interpretation of the tool call, interpretation and send back each time)

Again, a lot of improvements could be imagined there.

What we did in previous sections is reused.

# Setup

In [1]:
import os
from typing import Any
import re
import shutil

from docling.document_converter import DocumentConverter

from conversational_toolkit.vectorstores.chromadb import ChromaDBVectorStore
from conversational_toolkit.llms.base import LLMMessage, Roles
from conversational_toolkit.tools.base import Tool
from conversational_toolkit.agents.tool_agent import ToolAgent
from conversational_toolkit.agents.base import QueryWithContext
from conversational_toolkit.embeddings.openai import OpenAIEmbeddings
from conversational_toolkit.llms.openai import OpenAILLM
from conversational_toolkit.chunking.base import Chunk

from utils.specific_chunker import SpecificCharChunker

c:\Users\sieverin\SDSC\Code\sme-kt-zh-collaboration-rag\rag_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Consider using the pymupdf_layout package for a greatly improved page layout analysis.


In [3]:
path_to_docs = "data/docs"
path_to_document = os.path.join(path_to_docs, "alexnet_paper.pdf")

path_to_db = "data/db"
path_to_vectorstore = os.path.join(path_to_db, "example.db")

In [4]:
doc_converter = DocumentConverter()

conv_res = doc_converter.convert(path_to_document)
md = conv_res.document.export_to_markdown()

# replace \n per " ", as often just new lines
md = re.sub(r"(?<!\n)\n(?!\n)", " ", md)

doc_title_to_document = {"alexnet_paper.pdf": md}

chunker = SpecificCharChunker()
chunks = chunker.make_chunks(
    split_characters=["\n\n\n", "\n\n", "\n"],
    document_to_text=doc_title_to_document,
    max_number_of_characters=1024,
)

if os.path.exists(path_to_vectorstore):
    shutil.rmtree(path_to_vectorstore)
embedding_model = OpenAIEmbeddings(model_name="text-embedding-3-small")
embeddings = await embedding_model.get_embeddings([c.content for c in chunks])
vector_store = ChromaDBVectorStore(path_to_vectorstore)

await vector_store.insert_chunks(chunks=chunks, embedding=embeddings)

[INFO] 2026-02-26 15:21:31,565 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2026-02-26 15:21:31,570 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\sieverin\SDSC\Code\sme-kt-zh-collaboration-rag\rag_venv\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2026-02-26 15:21:31,575 [RapidOCR] main.py:53: Using C:\Users\sieverin\SDSC\Code\sme-kt-zh-collaboration-rag\rag_venv\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2026-02-26 15:21:31,659 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2026-02-26 15:21:31,661 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\sieverin\SDSC\Code\sme-kt-zh-collaboration-rag\rag_venv\Lib\site-packages\rapidocr\models\ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2026-02-26 15:21:31,661 [RapidOCR] main.py:53: Using C:\Users\sieverin\SDSC\Code\sme-kt-zh-collaboration-rag\rag_venv\Lib\site-packages\rapidocr\models\ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 202

In [5]:
def chunks_to_text(chunks: list[Chunk]) -> str:
    text = ""

    for chunk in chunks:
        text += (
            f"## Chunk {chunk.title}:\n```\n{chunk.content}\n```\n" + "-" * 30 + "\n\n"
        )

    text = text[:-4]

    return text


class RetrieveRelevantChunks(Tool):
    def __init__(
        self,
        name: str,
        description: str,
        parameters: dict[str, Any],
    ):
        self.name = name
        self.description = description
        self.parameters = parameters

    async def call(self, args: dict[str, Any]) -> dict[str, Any]:
        query = args.get("query")
        top_k = args.get("top_k", 5)

        if top_k > 10:
            raise ValueError("top_k cannot be greater than 10.")

        query_embedding = await embedding_model.get_embeddings([query])
        retrieved_chunks = await vector_store.get_chunks_by_embedding(
            embedding=query_embedding, top_k=top_k
        )

        retrieved_chunks_as_text = chunks_to_text(retrieved_chunks)

        return {"result": retrieved_chunks_as_text}


alexnet_retriever_tool = RetrieveRelevantChunks(
    name="retrieve_relevant_chunks",
    description="Retrieves the most relevant chunks from the AlexNet paper based on a query.",
    # What parameters it expects
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The query to retrieve relevant chunks for.",
            },
            "top_k": {
                "type": "number",
                "description": "The number of top relevant chunks to retrieve, maximum is 10.",
            },
        },
        "required": ["query"],
        "additionalProperties": False,
    },
)

# Define the Agent

In [6]:
class RAGAgent(ToolAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


llm = OpenAILLM(tool_choice="auto", tools=[alexnet_retriever_tool])

# Define the prompt
prompt = "You are a helpful assistant, answer shortly. Use the tools only when they are relevant, but if you do so, trust the results from the tools and use them in your answer, cite them precisely if you use them."
prompt_as_message = LLMMessage(content=prompt, role=Roles.SYSTEM)

2026-02-26 15:21:45.866 | DEBUG    | conversational_toolkit.llms.openai:__init__:63 - OpenAI LLM loaded: gpt-4o-mini; temperature: 0.5; seed: 42; tools: [<__main__.RetrieveRelevantChunks object at 0x000002854BA7B500>]; tool_choice: auto; response_format: {'type': 'text'}


In [7]:
rag_agent = RAGAgent(system_prompt=prompt, llm=llm, max_steps=5)

# Test the agent

## First Simple Question

In [8]:
conversation = [prompt_as_message]

In [9]:
query = "What is a CNN?"
query_as_message = LLMMessage(content=query, role=Roles.USER)
query_with_context = QueryWithContext(query=query, history=[])

answer = await rag_agent.answer(query_with_context)

2026-02-26 15:21:48.075 | DEBUG    | conversational_toolkit.agents.tool_agent:answer_stream:106 - [{'content': 'A Convolutional Neural Network (CNN) is a type of deep learning model specifically designed for processing structured grid data, such as images. CNNs use convolutional layers to automatically detect and learn features from the input data, reducing the need for manual feature extraction. They are particularly effective in image recognition, classification, and computer vision tasks due to their ability to capture spatial hierarchies in images. CNNs typically consist of convolutional layers, pooling layers, and fully connected layers.', 'tool_calls': [], 'role': <Roles.ASSISTANT: 'assistant'>, 'function_name': 'llm'}]


In [10]:
print(answer.content)

A Convolutional Neural Network (CNN) is a type of deep learning model specifically designed for processing structured grid data, such as images. CNNs use convolutional layers to automatically detect and learn features from the input data, reducing the need for manual feature extraction. They are particularly effective in image recognition, classification, and computer vision tasks due to their ability to capture spatial hierarchies in images. CNNs typically consist of convolutional layers, pooling layers, and fully connected layers.


In [11]:
conversation += [query_as_message, answer]

## Follow Up about AlexNet

And test remembers conversation.

In [12]:
query = "What are the top-1 and top-5 scores obtained on 'ILSVRC-2010' by AlexNet? How does AlexNet relate to my first question? Answer to both in few sentences."
query_as_message = LLMMessage(content=query, role=Roles.USER)
query_with_context = QueryWithContext(query=query, history=conversation)

answer = await rag_agent.answer(query_with_context)

2026-02-26 15:21:51.268 | INFO     | conversational_toolkit.embeddings.openai:get_embeddings:38 - OpenAI embeddings shape: (1, 1024)
2026-02-26 15:21:51.504 | INFO     | conversational_toolkit.embeddings.openai:get_embeddings:38 - OpenAI embeddings shape: (1, 1024)
2026-02-26 15:21:55.161 | DEBUG    | conversational_toolkit.agents.tool_agent:answer_stream:106 - [{'content': '', 'tool_calls': [ToolCall(id='call_qO0qnLW4YVkyR3TgeoKNRO4Z', function=Function(name='retrieve_relevant_chunks', arguments='{"query": "ILSVRC-2010 AlexNet top-1 top-5 scores"}'), type='function'), ToolCall(id='call_OUcCBcYCuSl88H4jEQKX5Y9K', function=Function(name='retrieve_relevant_chunks', arguments='{"query": "AlexNet CNN"}'), type='function')], 'role': <Roles.ASSISTANT: 'assistant'>, 'function_name': 'llm'}, {'result': "## Chunk 15:\n```\nILSVRC-2010 is the only version of ILSVRC for which the test set labels are available, so this is the version on which we performed most of our experiments. Since we also ent

In [13]:
print(answer.content)

AlexNet achieved a top-1 error rate of 37.5% and a top-5 error rate of 17.0% on the ILSVRC-2010 dataset, significantly outperforming previous models at the time. This performance was a result of its deep convolutional architecture, which includes five convolutional layers and three fully connected layers, leveraging techniques like dropout for regularization and efficient GPU implementations for training.

AlexNet is a specific type of Convolutional Neural Network (CNN), designed for image classification tasks, which explains its relevance to your first question about CNNs. Its architecture and training methods have had a substantial impact on the field of deep learning and computer vision.


----------------